In [5]:
import os
from PIL import Image, ImageDraw, ImageFont
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4

# Generate the required numbers based on the specified rules
def generate_number_list():
    numbers = []
    # One instance of 2 and 12
    numbers.append(2)
    numbers.append(12)
    
    # Two instances of 3 to 6 and 8 to 11
    for num in range(3, 7):
        numbers.extend([num, num])
    for num in range(8, 12):
        numbers.extend([num, num])
    
    return numbers

# Create 1-inch diameter circle images with customizable text size and colors
def create_circle_image_with_number(number, diameter_in_inches=1, dpi=300, font_size=50, circle_color="#FFFFFF", text_color="#000000"):
    diameter = int(diameter_in_inches * dpi)  # 1 inch at 300 DPI
    img = Image.new("RGB", (diameter, diameter), circle_color)
    draw = ImageDraw.Draw(img)
    
    # Draw a circle outline
    draw.ellipse([(0, 0), (diameter-1, diameter-1)], outline="black", width=5)
    
    # Load a fancy font suitable for board games
    font = ImageFont.truetype("Lobster-Bold.ttf", font_size)  # Example font; ensure it's available or use another
    
    # Convert number to string and calculate text size
    text = str(number)
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    
    text_x = (diameter - text_width) / 2
    text_y = (diameter - text_height) / 2 - (0.3 * text_height)  # Move text slightly up
    
    # Draw the number with the specified color
    draw.text((text_x, text_y), text, fill=text_color, font=font)
    
    return img

# Save circle images to a PDF with cleanup of temporary PNGs
def save_circles_to_pdf(numbers, output_pdf="output_numbers.pdf", font_size=50, circle_color="#FFFFFF", text_color="#000000"):
    c = canvas.Canvas(output_pdf, pagesize=A4)
    dpi = 300
    inch_to_pt = 72  # 1 inch = 72 points in PDF
    
    # Set page size and circle size
    page_width, page_height = A4
    circle_diameter_in_pts = 1 * inch_to_pt
    
    # Calculate max rows and columns for 1 inch circles on A4 page
    max_cols = int(page_width // circle_diameter_in_pts)
    max_rows = int(page_height // circle_diameter_in_pts)
    
    # Draw each circle on the PDF
    for i, number in enumerate(numbers):
        img = create_circle_image_with_number(number, font_size=font_size, circle_color=circle_color, text_color=text_color)
        x_pos = (i % max_cols) * circle_diameter_in_pts
        y_pos = page_height - ((i // max_cols) + 1) * circle_diameter_in_pts
        
        img_path = f"{number}_{i}.png"
        img.save(img_path)  # Save the circle temporarily
        
        # Draw the image on the PDF
        c.drawImage(img_path, x_pos, y_pos, width=circle_diameter_in_pts, height=circle_diameter_in_pts)
        
        if (i + 1) % (max_rows * max_cols) == 0:  # Start a new page if needed
            c.showPage()

    c.save()

    # Cleanup: remove temporary PNG files
    for i, number in enumerate(numbers):
        img_path = f"{number}_{i}.png"
        if os.path.exists(img_path):
            os.remove(img_path)

# Example use with numbers following the specified rules
numbers = generate_number_list()
save_circles_to_pdf(numbers, font_size=185, circle_color="#FFDDC1", text_color="#000000")
